In [1]:
##밀고 당기기 게임
import pandas as panda


class Game:
    def __init__(self, players_, actions_, payoffs_):
        self.players = players_
        self.actions = actions_
        self.payoffs = payoffs_      

HC_Game = Game(["Player1", "Player2"],{"Push":0, "Stall":1, "Pull":2} ,
               [[(-5,-5), (0,-1), (3,-3)], [(-1,0),(0,0),(1,0)], [(-3,3), (0,1), (2,2)]])




In [2]:
panda.DataFrame(HC_Game.payoffs)

,0,1,2
0,"(-5, -5)","(0, -1)","(3, -3)"
1,"(-1, 0)","(0, 0)","(1, 0)"
2,"(-3, 3)","(0, 1)","(2, 2)"


In [3]:
import random

def StateRandGenerate():
    index = random.randrange(0,3)
    if index == 0:
        return "Push"
    elif index == 1:
        return "Stall"
    else: 
        return "Pull" 

StateRandGenerate()

'Stall'

In [4]:
import math
class PlayerModel_DFA:
    def __init__(self, dfa, start):
        self.dfa = dfa
        self.start = start
        
dfa = {"Push":{'Push':'Push', 'Stall':"Stall", "Pull":"Pull"},
       "Stall":{'Push':'Pull', 'Stall':'Push', "Pull":"Pull"},
       "Pull":{'0':'Defer', 'S':'Pull', "Pull":"Pull"}}

##The Structure of Automata
##{CurrentState :{..., Input : NextState ,...}}


###DFA of player models
AllPull = {"Pull":{"Pull":"Pull", "Stall":"Pull", "Push":"Pull"}}
AllStall = {"Stall":{"Pull":"Stall", "Stall":"Stall", "Push":"Stall"}}
AllPush = {"Push":{"Pull":"Push", "Stall":"Push", "Push":"Push"}}
TitForTat = {
    "Stall" : {"Pull":"Pull", "Stall":"Stall", "Push":"Push"},
    "Push" : {"Pull":"Pull", "Stall":"Stall", "Push":"Push"},
    "Pull" : {"Pull":"Pull", "Stall":"Stall", "Push":"Push"}
}
Patient = {
    "Pull" : { "Pull":"Pull", "Stall":"Stall", "Push":"Stall" },
    "Stall" : {"Pull" : "Pull", "Stall" : "Stall", "Push":"Push"},
    "Push" : {"Pull": "Push", "Stall" : "Push", "Push" : "Push"}
}

###Automata Player Models
Player_AllPull = PlayerModel_DFA(AllPull, "Pull")
Player_AllStall = PlayerModel_DFA(AllStall, "Stall")
Player_AllPush = PlayerModel_DFA(AllPush, "Push")
Player_TitForTat = PlayerModel_DFA(TitForTat, "Pull")
Player_Patient = PlayerModel_DFA(Patient, "Pull")

List_Players = [Player_AllPull, Player_AllStall, Player_AllPush, Player_TitForTat, Player_Patient]

# Player1 = PlayerModel_DFA(dfa, "Push")
# Player2 = PlayerModel_DFA(dfa, "Defer")

def PlayGame(Game, Player1, Player2, iterate, W):
    ##Init Player1,2 Score
    Player1_Score = 0
    Player2_Score = 0
    
    #Next State
    Player1_Current_State = Player1.start
    Player2_Current_State = Player2.start
        
    first_game = Game.payoffs[Game.actions[Player1_Current_State]][Game.actions[Player2_Current_State]]
    Player1_Score = Player1_Score + first_game[0]
    Player2_Score = Player1_Score + first_game[1]
    for i in range(iterate-1):
        Save_Player1 = Player1_Current_State
        Save_Player2 = Player2_Current_State
        
        Player1_Current_State = Player1.dfa[Save_Player1][Save_Player2]
        Player2_Current_State = Player2.dfa[Save_Player2][Save_Player1]
        
        next_game = Game.payoffs[Game.actions[Player1_Current_State]][Game.actions[Player2_Current_State]]
        
        ##Update Summation
        Player1_Score = Player1_Score + math.pow(W, i+1)*next_game[0]
        Player2_Score = Player2_Score + math.pow(W, i+1)*next_game[1]
    return (Player1_Score, Player2_Score)



In [5]:
def PlayGame_Random(Game, PlayerOpp, iterate):
    PlayerRandomScore = 0
    
    PlayerRandom_Current_State = StateRandGenerate()
    PlayerOpp_Current_State = PlayerOpp.start
    
    
    first_game = Game.payoffs[Game.actions[PlayerRandom_Current_State]][Game.actions[PlayerOpp_Current_State]]
    PlayerRandomScore = PlayerRandomScore + first_game[0]
    for i in range(iterate-1):
        Save_PlayerRandom = PlayerRandom_Current_State 
        Save_PlayerOpp_Current_State = PlayerOpp_Current_State
        
        PlayerRandom_Current_State =StateRandGenerate()
        PlayerOpp_Current_State = PlayerOpp.dfa[Save_PlayerRandom][Save_PlayerOpp_Current_State]
        
        ##Update Summation
        next_game = Game.payoffs[Game.actions[PlayerRandom_Current_State]][Game.actions[PlayerOpp_Current_State]]
        PlayerRandomScore = PlayerRandomScore + next_game[0]
    return PlayerRandomScore

# print(PlayGame_Random(HC_Game, Player_TitForTat, 10))


In [6]:
##Result of All Pull VS Others

Result = 0

##With Automata
for i in range(len(List_Players)):
    Result = Result + PlayGame(HC_Game, Player_AllPull, List_Players[i], 50, 0.75)[0]
print(Result)

11.999993204140122


In [7]:
##Result of All Stall VS Others

Result = 0

##With Automata
for i in range(len(List_Players)):
    Result = Result + PlayGame(HC_Game, Player_AllStall, List_Players[i], 50, 0.75)[0]
print(Result)

2.0


In [8]:
##Result of All Push VS Others

Result = 0

##With Automata
for i in range(len(List_Players)):
    Result = Result + PlayGame(HC_Game, Player_AllPush, List_Players[i], 50, 0.75)[0]
print(Result)

-28.2499728165605


In [9]:
##Result of TitForTat VS Others

Result = 0

##With Automata
for i in range(len(List_Players)):
    Result = Result + PlayGame(HC_Game, Player_TitForTat, List_Players[i], 50, 0.75)[0]
print(Result)

5.9999977347133715


In [10]:
##Result of BePatient VS Others

Result = 0

##With Automata
for i in range(len(List_Players)):
    Result = Result + PlayGame(HC_Game, Player_Patient, List_Players[i], 50, 0.75)[0]
print(Result)

8.999997734713364
